In [142]:
#!pip install openpyxl


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [148]:
pmep = pd.read_excel('MEPS 2023 Prescribed Medication.xlsx')

pmep_filtered = pmep[['DUPERSID', 'RXNAME', 'RXNDC', 'RXQUANTY', 'RXSF23X', 'PERWT23F', 'RXMR23X' ]]

pmep_filtered = pmep_filtered.rename(columns={
    'DUPERSID': 'DRUG_ID',
    'RXNAME' : 'RX_NAME',
    'RXNDC' : 'NDC',
    'RXQUANTY' : 'RX_QUANTITY',
    'RXSF23X' : 'OOP_COST',
    'PERWT23F' : 'SURVEY_WEIGHT',
    'RXMR23X' : 'MEDICARE_AMT'
})

In [151]:
consolidated_cols = [
    "DUPERSID",   # drug key
    "POVCAT23",   # income category
    "RACEV2X",    # race/ethnicity
    "AGE23X",     # age
    "SEX",        # sex
    "MCREV23",    # Medicare enrollment
    "REGION23",   #census region
    "TTLP23X"     #total income
]

consolidated_filtered = pd.read_excel(
    "MEPS 2023 Full Consolidated.xlsx",
    usecols=consolidated_cols
)

consolidated_medicare = consolidated_filtered[consolidated_filtered['MCREV23'] == 1]

In [154]:
consolidated_medicare = consolidated_medicare.rename(columns={
    "DUPERSID": "DRUG_ID",
    "POVCAT23": "INCOME_CATEGORY",
    "RACEV2X": "RACE",
    "AGE23X": "AGE",
    "SEX": "SEX",
    "MCREV23": "HAS_MEDICARE",
    "REGION23": "CENSUS_REGION",
    "TTLP23X": "TOTAL_INCOME"
})

In [157]:
merged_meps = pd.merge(
    pmep_filtered,
    consolidated_medicare,
    on='DRUG_ID',
    how='inner'
)

In [160]:
merged_meps.isnull().sum()

DRUG_ID            0
RX_NAME            0
NDC                0
RX_QUANTITY        0
OOP_COST           0
SURVEY_WEIGHT      0
MEDICARE_AMT       0
CENSUS_REGION      0
AGE                0
SEX                0
RACE               0
TOTAL_INCOME       0
INCOME_CATEGORY    0
HAS_MEDICARE       0
dtype: int64

In [163]:
# List of special MEPS codes to remove
invalid_codes = [-1, -2, -7, -8, -10, -15]


cols_to_check = [
    'OOP_COST',       # prescription out-of-pocket
    'AGE',            # patient age
    'INCOME_CATEGORY',# income category
    'RACE',           # race/ethnicity
    'SEX',            # sex
    'HAS_MEDICARE',   # Medicare flag
    'TOTAL_INCOME',   # total household income
    'CENSUS_REGION'   # census region
]


merged_meps = merged_meps[~merged_meps[cols_to_check].isin(invalid_codes).any(axis=1)]

In [166]:
region_map = {
    1: "Northeast",
    2: "Midwest",
    3: "South",
    4: "West"
}

# Apply mapping
merged_meps['CENSUS_REGION'] = merged_meps['CENSUS_REGION'].map(region_map)

sex_map = {
    1: "Male",
    2: "Female"
}

# Apply mapping
merged_meps['SEX'] = merged_meps['SEX'].map(sex_map)

race_map = {
    1: "White",
    2: "Black",
    3: "American Indian/Alaska Native",
    4: "Asian Indian",
    5: "Chinese",
    6: "Filipino",
    10: "Other Asian/NH/PI",
    12: "Multiple races",
    -1: "Inapplicable"
}

# Apply mapping
merged_meps['RACE'] = merged_meps['RACE'].map(race_map)


income_mapping = {
    1: 'Poor',
    2: 'Near Poor',
    3: 'Low Income',
    4: 'Middle Income',
    5: 'High Income'
}

# Apply mapping
merged_meps['INCOME_CATEGORY'] = merged_meps['INCOME_CATEGORY'].map(income_mapping)

In [169]:
def recode_race(code):
    if code == "White":
        return "White"
    elif code == "Black":
        return "Black"
    elif code in ["Asian Indian", "Chinese", "Filipino"]:
        return "Asian"
    elif code in ["American Indian/Alaska Native", "Other Asian/NH/PI"]:
        return "Other"
    elif code == "Multiple races":
        return "Multiple Races"
    else:
        return None  


merged_meps['RACE_GROUP'] = merged_meps['RACE'].apply(recode_race)

print(merged_meps['RACE_GROUP'].value_counts())

RACE_GROUP
White             83673
Black             14011
Multiple Races     2424
Other              2206
Asian              1772
Name: count, dtype: int64


In [175]:
top10_drugs = (
    merged_meps.groupby('RX_NAME')
    .size()  
    .nlargest(10)
    .index
)

top10_data = merged_meps[merged_meps['RX_NAME'].isin(top10_drugs)]

In [178]:
#!pip install statsmodels


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [193]:
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols


race_groups = top10_data['RACE_GROUP'].unique()

results = []

for race in race_groups:
    group = top10_data[top10_data['RACE_GROUP'] == race]['OOP_COST']
    
    others = top10_data[top10_data['RACE_GROUP'] != race]['OOP_COST']
    
    t_stat, p_value = stats.ttest_ind(group, others, equal_var=False)
    
    results.append({
        'Race': race,
        'Group Mean OOP': group.mean(),
        'T-Statistic': t_stat,
        'P-value': round(p_value, 6)
    })


results_df = pd.DataFrame(results)

def format_pvalue(p):
    if p < 0.0001:
        return "<0.0001"
    else:
        return f"{p:.5f}"

results_df['P-value'] = results_df['P-value'].apply(format_pvalue)

results_df = results_df.sort_values(by='T-Statistic').reset_index(drop=True)


def bold_races(val):
    if val in ["Black", "Multiple Races", "Other", "White"]:
        return 'font-weight: bold'
    return ''

styled = results_df.style \
    .format({"Group Mean OOP": "{:.2f}", "T-Statistic": "{:.3f}"}) \
    .set_caption("OOP Cost Comparison by Race") \
    .hide(axis="index") \
    .background_gradient(subset=["T-Statistic"], cmap="coolwarm") \
    .applymap(bold_races, subset=['Race']) 

styled

/tmp/ipykernel_941/2641996864.py:48: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled = results_df.style \


Race,Group Mean OOP,T-Statistic,P-value
Black,3.45,-11.590,<0.0001
Multiple Races,3.83,-3.202,0.00143
Asian,4.72,-0.086,0.93172
Other,6.68,4.737,<0.0001
White,4.94,7.653,<0.0001


In [196]:
income_groups = top10_data['INCOME_CATEGORY'].unique()

results = []

for group in income_groups:
    group_data = top10_data[top10_data['INCOME_CATEGORY'] == group]['OOP_COST']
    other_data = top10_data[top10_data['INCOME_CATEGORY'] != group]['OOP_COST']
    
    t_stat, p_value = stats.ttest_ind(group_data, other_data, equal_var=False, nan_policy='omit')
    
    mean_group = group_data.mean()
    
    results.append({
        'Income Group': group,
        'Group Mean OOP': mean_group,
        'T-Statistic': t_stat,
        'P-value': p_value
    })

results_df = pd.DataFrame(results)

results_df['Group Mean OOP'] = results_df['Group Mean OOP'].round(2)
results_df['T-Statistic'] = results_df['T-Statistic'].round(3)

def format_pvalue(p):
    if p < 0.0001:
        return "<0.0001"
    else:
        return f"{p:.5f}"

results_df['P-value'] = results_df['P-value'].apply(format_pvalue)


results_df = results_df.sort_values(by='T-Statistic').reset_index(drop=True)

def bold_all(val):
    return 'font-weight: bold'

styled = results_df.style \
    .format({"Group Mean OOP": "{:.2f}", "T-Statistic": "{:.3f}"}) \
    .set_caption("OOP Cost Comparison by Income Group") \
    .hide(axis="index") \
    .background_gradient(subset=["T-Statistic"], cmap="coolwarm") \
    .applymap(bold_all, subset=['Income Group'])  

styled

/tmp/ipykernel_941/2256836279.py:46: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled = results_df.style \


Income Group,Group Mean OOP,T-Statistic,P-value
Poor,2.73,-21.541,<0.0001
Near Poor,3.08,-10.406,<0.0001
Low Income,3.71,-8.950,<0.0001
Middle Income,5.31,6.242,<0.0001
High Income,6.20,18.969,<0.0001


In [199]:
regions = top10_data['CENSUS_REGION'].unique()

results = []

for region in regions:
    region_data = top10_data[top10_data['CENSUS_REGION'] == region]['OOP_COST']
    other_data = top10_data[top10_data['CENSUS_REGION'] != region]['OOP_COST']
    
    t_stat, p_value = stats.ttest_ind(region_data, other_data, equal_var=False, nan_policy='omit')
    mean_region = region_data.mean()
    
    results.append({
        'Region': region,
        'Group Mean OOP': mean_region,
        'T-Statistic': t_stat,
        'P-value': p_value
    })


results_df = pd.DataFrame(results)

results_df['Group Mean OOP'] = results_df['Group Mean OOP'].round(2)
results_df['T-Statistic'] = results_df['T-Statistic'].round(3)

def format_pvalue(p):
    if p < 0.0001:
        return "<0.0001"
    else:
        return f"{p:.5f}"

results_df['P-value'] = results_df['P-value'].apply(format_pvalue)


results_df = results_df.sort_values(by='T-Statistic').reset_index(drop=True)

def bold_all(val):
    return 'font-weight: bold'

styled = results_df.style \
    .format({"Group Mean OOP": "{:.2f}", "T-Statistic": "{:.3f}"}) \
    .set_caption("OOP Cost Comparison by Census Region") \
    .hide(axis="index") \
    .background_gradient(subset=["T-Statistic"], cmap="coolwarm") \
    .applymap(bold_all, subset=['Region'])  

styled

/tmp/ipykernel_941/1607752319.py:44: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled = results_df.style \


Region,Group Mean OOP,T-Statistic,P-value
South,4.33,-6.567,<0.0001
Northeast,4.54,-2.038,0.04157
West,5.15,3.854,0.00012
Midwest,5.25,4.696,<0.0001


In [ ]:
merged_meps['oop_pct_income'] = (merged_meps['OOP_COST'] / (merged_meps['TOTAL_INCOME']/12)) * 100

hardship_data = merged_meps[(merged_meps['TOTAL_INCOME'] > 0) & 
                       (merged_meps['OOP_COST'] > 0) &  
                       (merged_meps['oop_pct_income'] < 50)] 

# Calculate average burden by income group
avg_burden = hardship_data.groupby('INCOME_LABEL')['oop_pct_income'].agg(['mean', 'median']).reset_index()
avg_burden = avg_burden.sort_values('mean', ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

x = np.arange(len(avg_burden))
bars = ax.bar(x, avg_burden['mean'], alpha=0.8, edgecolor='black', linewidth=1.5)

ax.set_xlabel('Income Level')
ax.set_ylabel('OOP as % of Monthly Family Income')  
ax.set_title('Financial Burden: OOP per Prescription as % of Monthly Income\n(Prescriptions with Out-of-Pocket Costs Only)', 
             fontweight='bold', pad=20)

ax.set_xticks(x)
ax.set_xticklabels(avg_burden['INCOME_LABEL'], fontsize=12)
ax.grid(axis='y', alpha=0.3, linestyle='--')


for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}%', 
            ha='center', va='bottom', fontweight='bold', fontsize=11)


plt.tight_layout()
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=af057222-3fdc-4af3-880a-2119daf62f90' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>